In [12]:
# Imports
import json
import numpy as np
import math

In [13]:
# Define the square in which the analyzed area is included, and the step - size of the rectangles 
# (e.g., with step = 0.001, a square latitude would be for example [8.4521, 8.4531])
'''
#Bern
start_y = 46.9193
end_y = 46.9672
start_x = 7.3662
end_x = 7.4873
step = 0.0005
'''
'''
#Zurich
start_y = 47.3179
end_y = 47.4265
start_x = 8.4521
end_x = 8.6126
step = 0.001
'''
#Frauenfeld
start_y = 47.5321
end_y = 47.5768
start_x = 8.8431
end_x = 8.9452
step = 0.001

In [14]:
# Use decimal library to avoid problems with float imprecision in computations
import decimal

# Drange is doing the same as np.arange, but without float imprecision
def drange(x, y, jump):
    x = decimal.Decimal(x)
    y = decimal.Decimal(y)
    while x < y:
        yield float(x)
        x += decimal.Decimal(jump)

In [15]:
# Initialize dictionary with squares tesselating the whole area between the lat and long selected before
squares = {}
for y in drange(str(start_y), str(end_y), str(step)):
    for x in drange(str(start_x), str(end_x), str(step)):
        squares[f"{y};{x}"] = []

In [16]:
# Load file containing points, to be put in the right tesselation
with open("safety_scores/frauenfeld.json") as f:
    data = json.load(f)

In [17]:
# For each square in the tesselation, create a list of all scores of the points in that square
for f in data["features"]:
    y = np.floor((decimal.Decimal(f'{f["geometry"]["coordinates"][1]:.3f}')-decimal.Decimal(str(start_y)))/decimal.Decimal(str(step)))*decimal.Decimal(str(step))+decimal.Decimal(str(start_y))
    x = np.floor((decimal.Decimal(f'{f["geometry"]["coordinates"][0]:.3f}')-decimal.Decimal(str(start_x)))/decimal.Decimal(str(step)))*decimal.Decimal(str(step))+decimal.Decimal(str(start_x))
    if str(y)+";"+str(x) in squares:
        squares[str(y)+";"+str(x)].append(f["properties"]["score"])

In [18]:
# Create geojson with the tesselation and the score for each square (average of all points contained in it)
geojson = {"type" : "FeatureCollection",
          "features" : []
          }
for k,v in squares.items():
    t = k.split(";")
    y, x = float(t[0]), float(t[1])
    
    if len(v) > 0:
        geojson["features"].append({
          "type": "Feature",
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                  [x,y], [x+step, y], [x+step, y+step], [x, y+step]
              ]
            ]
          },
          "properties": {
              "score": int(np.mean(v))
          }
        })

In [19]:
# Output to json file
with open("frauenfeld_tesselated_0.001.json", 'w') as outfile:
    json.dump(geojson, outfile)